In [1]:
from time import sleep
import pandas as pd
import requests
import json

def get_api(address):
  x = requests.get('http://api-ropsten.etherscan.io/api?module=account&action=tokentx&address='+address+'&startblock=0&endblock=999999999&sort=asc&apikey=FV2TJTM9U8WYF4TA56DE9UIBYN4ZGWM198', headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'})
  return json.loads(x.content)

In [2]:
df = pd.DataFrame([], columns=['hash','from','to','value'])
user_address = '0xEcA19B1a87442b0c25801B809bf567A6ca87B1da'
user_transactions = [t for t in get_api(user_address)['result'] if t['tokenSymbol'] == 'BKTC' and t['from'] == user_address.lower()]
user_transactions = [(t['hash'], t['from'], t['to'], t['value'], t['timeStamp']) for t in user_transactions]
df = df.append(pd.DataFrame(user_transactions, columns=['hash','from','to','value','timestamp']))

In [3]:
check = 0
cur_list = ['check']
while len(cur_list) > 0:
  cur_list = []
  tmp = df.copy()
  for i in range(check, len(tmp)):
    address = df['to'][i]
    time = df['timestamp'][i]
    user_transactions = []
    for t in get_api(address)['result']: 
      if type(t) == str:
        continue
      if t['tokenSymbol'] == 'BKTC' and t['from'] == address.lower() and t['timeStamp'] > time:
        user_transactions.append(t)
    user_transactions = [(t['hash'], t['from'], t['to'], t['value'], t['timeStamp']) for t in user_transactions]
    cur_list = cur_list + user_transactions
    df = df.append(pd.DataFrame(user_transactions, columns=['hash','from','to','value','timestamp'])).reset_index(drop=True)
    check = len(tmp)
    sleep(1)

In [4]:
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.iloc[:,:-1]
df['value'] = pd.to_numeric(df['value'])

In [5]:
user_list = []
user_list = user_list + df['from'].tolist()
user_list = user_list + df['to'].tolist()
user_list = list(set(user_list))

In [6]:
res = []
for user in user_list:
  address = user
  cal = 0
  for t in get_api(address)['result']:
    if t['tokenSymbol'] == 'BKTC' and t['from'] == address:
      cal -= int(t['value'])
  sleep(1)
  for t in get_api(address)['result']:
    if t['tokenSymbol'] == 'BKTC' and t['to'] == address:
      cal += int(t['value'])
  sleep(1)
  res.append((user, cal))

In [7]:
res = pd.DataFrame(res, columns=['address', 'amount'])

In [8]:
df.to_csv('transaction.csv')
res.to_csv('amount.csv')